### In this hands-on you will be using pretrained yolo model for object detection.
#### Follow thr instructions provided for each cell and code accordingly.
#### Press shift + enter to run the in each cell and make sure you do this for all the cells in the notebook.

### Run the below cell to import all necessary packages required for this hands-on

In [17]:
!alias python=python3
!alias pip=pip3
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from keras.models import load_model, Model
from keras import backend as K
import tensorflow as tf

from utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body


#### compute box_scores by multiplying box_confidence and box_class_probs  
#### extracts class indices of maximum score and their corresponding score values and assign it to variable box_classes and box_class_scores  
#### filter out the box_scores above a threshold, and return the corresponding scores, boxe dimensions and their classes.

In [18]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.6):
    
    
    box_scores =              
    box_classes = 
    box_class_scores =
    
    filtering_mask = 
    
    scores = 
    boxes = 
    classes = 
    
    return scores, boxes, classes

#### Run the below cell to test the function defined above
### Expected output:
scores[2] = 18.66819   
boxes[2] = [ 1.9018091 -3.9903622 -4.009363  -3.981121 ]  
classes[2] = 21  
scores.shape = (?,)  
boxes.shape = (?, 4)  
classes.shape = (?,)  


In [ ]:
with tf.Session() as test_a:
    box_confidence = tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 51)
    boxes = tf.random_normal([19, 19, 5, 4], mean=1, stddev=4, seed = 51)
    box_class_probs = tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 51)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

#### Define the function yolo_non_max_supression to implement nonmax suppression to eliminate overlapping boxes.



In [21]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    
    
    max_boxes_tensor =    # tensor to be used in tf.image.non_max_suppression()
    
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    nms_indices =      ###use tf.image.non_max_suppression to extract eliminate ovelapping boxes
    ####Use tf.gather() to pick boxes and their respective predicted classes as well as their scores
    scores =           
    boxes = 
    classes = 
    
    
    return scores, boxes, classes

#### Run the below cell to validate the function defined above  
#### Expected output
scores[2] = 7.2168345  
boxes[2] = [  1.3835918  -2.2073252   2.5128307 -10.085559 ]  
classes[2] = -5.1675873  
scores.shape = (10,)  
boxes.shape = (10, 4)  
classes.shape = (10,)  

In [ ]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 51)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 51)
    classes = tf.random_normal([54,], mean=1, stddev=4, seed = 51)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
    init=tf.global_variables_initializer()
    test_b.run(init)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

    

#### Define the function yolo_eval that consolidates all the function defined above to extract single prediction boxes for each object

In [23]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=5, score_threshold=.6, iou_threshold=.5):
    
    box_xy, box_wh,  box_confidence, box_class_probs = yolo_outputs   
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    scores, boxes, classes =    ###call yolo_filter_boxes to extract box confidence scores
    
    ### the scale_box function adjest the box dimensions according to the image.
    boxes = scale_boxes(boxes, image_shape)  

    scores, boxes, classes =  ### call yolo_non_max_suppression to eliminate overlapping boxes
    return scores, boxes, classes

#### Run the below cell to vlidate yolo_eval function defined above
#### Expected output
scores[2] = 155.1541  
boxes[2] = [   109.86103  -5404.5874     329.5831  -16213.762  ]  
classes[2] = 16  
scores.shape = (5,)  
boxes.shape = (5, 4)  
classes.shape = (5,)  



In [ ]:
with tf.Session() as test_b:
    yolo_outputs = (tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 51),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 51),
                    tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 51),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 51))
    scores, boxes, classes = yolo_eval(yolo_outputs)
    init=tf.global_variables_initializer()
    test_b.run(init)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))


#### Run the below cell to load the model 

In [54]:
sess = K.get_session()
class_names = read_classes("model_data/pascal_classes.txt")
anchors = read_anchors("model_data/yolo_tiny_anchors.txt")
image_shape = (375., 500.)
yolo_model = load_model("model_data/yolo_tiny.h5")
yolo_model.summary()

### Extract predictions

In [56]:

yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names)) #### yolo_head() return box scores, dimensions and claases for each grid cell
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape) ### Filter boxes

#### The method to compute the prediction on an image file has been define for you

In [58]:
def predict(sess, image_file):
   

    image, image_data = preprocess_image(image_file, model_image_size = (416, 416))


    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict={yolo_model.input: image_data, K.learning_phase(): 0})
    

    
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))

    colors = generate_colors(class_names)
    
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    
    image.save(os.path.join("out", image_file), quality=90)
    
    output_image = scipy.misc.imread(os.path.join("out", image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes

### Run the below cell to detect objects on an image file

In [ ]:
predict(sess, "men.JPEG")